In [2]:
# ✅ WHOLE PROFESSIONAL VERSION of the Upper Limits Plotting Pipeline with improved aesthetics and structure

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from libraries import dataframe_generator

# --- Configuration ---
bin_size = 0.3
spectral_index = 2.07
PATH_BASE = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/'
GRB_KN_DIR = PATH_BASE
GITLAB_DIR = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GitLab_kn_paper/'
sep='pseudo'
config = {
    'GRBsINFO': PATH_BASE+'data/ULs/config/GRB_List.csv',
    'Franceschini_1st': PATH_BASE+f'data/ULs/files/PSF_0.3/alfa=2.07/{sep}/UpperLimit_1_Franceschini08.csv',
    'Gilmore_1st': PATH_BASE+f'data/ULs/files/PSF_0.3/alfa=2.07/{sep}/UpperLimit_1_Gilmore12Fiducial.csv',
    'Franceschini_2nd': PATH_BASE+f'data/ULs/files/PSF_0.3/alfa=2.07/{sep}/UpperLimit_2_Franceschini08.csv',
    'Gilmore_2nd': PATH_BASE+f'data/ULs/files/PSF_0.3/alfa=2.07/{sep}/UpperLimit_2_Gilmore12Fiducial.csv',
    'spectral_index': 2.07,
    'plots_dir': PATH_BASE+'data/ULs/plots/'
}


# upper_limits_plot.py

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

# ===========================
# 1. DATAFRAME GENERATOR
# ===========================
def DATAFRAME_generator(franceschini_file, gilmore_file, grb_config_file):
    config = pd.read_csv(grb_config_file)
    config['Zenith'] = config['Dec'] - 19

    if 'Name' not in config.columns:
        raise KeyError(f"[ERROR] La columna 'Name' no está en {grb_config_file}. Columnas disponibles: {config.columns.tolist()}")

    config['Name'] = config['Name'].str.replace(',', '').str.strip()
    config.sort_values(by='Name', inplace=True)
    config.reset_index(drop=True, inplace=True)

    columns = ['Name', 'Spectrum', 'TS', 'sqrt(TS)', 'flux', 'lowerErr', 'upperErr', 'lowerBound', 'upperBound']
    df_fra = pd.read_csv(franceschini_file, delim_whitespace=True, names=columns)
    df_gil = pd.read_csv(gilmore_file, delim_whitespace=True, names=columns)

    for df in [df_fra, df_gil]:
        df['Name'] = df['Name'].str.replace(',', '').str.strip()
        df.drop_duplicates(inplace=True)
        df.sort_values(by='Name', inplace=True)

    df_all = pd.merge(df_fra, df_gil, on='Name', suffixes=('_fra', '_gil'))
    df_all = pd.merge(df_all, config, on='Name', how='inner')
    df_all = df_all.loc[:, ~df_all.columns.duplicated()]

    return df_all

# ===========================
# 2. CONVERSIÓN DE FLUJOS
# ===========================
def non_to_int2(E1, E2, flux, alpha, pivot):
    A = flux / (pivot ** (-alpha))
    if alpha > 2:
        return 1.6 * A * ((E2 ** (2 - alpha)) - (E1 ** (2 - alpha))) / (2 - alpha)
    elif alpha == 2:
        return A * (np.log(E2) - np.log(E1))
    else:
        return None

# ===========================
# 3. FUNCIÓN DE GRAFICADO
# ===========================
def plot_ul(names, dec, ul_franceschini, ul_gilmore, color_hex,
            save_path_png=None, save_path_pdf=None, label=''):

    fig, ax = plt.subplots(figsize=(11, 7))
    plt.rcParams.update({"text.usetex": False, "font.family": "serif", "font.size": 14})

    legend_labels = []
    n_valid_points = 0
    valid_mask = (
        ~pd.isna(ul_gilmore) & ~pd.isna(ul_franceschini) & ~pd.isna(dec) &
        (ul_gilmore > 0) & (ul_franceschini > 0)
    )

    for grb, ul_f, ul_g, d, col in zip(names[valid_mask], ul_franceschini[valid_mask],ul_gilmore[valid_mask], dec[valid_mask], color_hex):
        ax.errorbar(x=d, y=ul_g, yerr=ul_g / 10, uplims=True,
                    color=col, linewidth=1.6, capsize=4, linestyle='solid')
        ax.fill_between([d - 0.4, d + 0.4], [ul_f, ul_f], [ul_g, ul_g],
                        color=col, alpha=0.5)
        legend_labels.append(grb)
        n_valid_points += 1
        print(grb,ul_f,ul_g)
    print(f"\U0001F4CA {label} Transit: {n_valid_points} GRBs plotted.")

    ax.set_yscale('log')
    ax.set_xlabel('Declination [deg]', fontsize=10, labelpad=10)
    ax.set_ylabel(r'Integrated Upper Limit $\Phi$ [erg cm$^{-2}$ s$^{-1}$]', fontsize=10, labelpad=10)
    ax.tick_params(axis='both', which='major', labelsize=14, direction='in', length=6, width=1.2)
    ax.tick_params(axis='both', which='minor', direction='in', length=4, width=1.0)
    ax.grid(True, which='both', linestyle=':', linewidth=0.8, alpha=0.6)
    for spine in ax.spines.values():
        spine.set_linewidth(1.2)

    # ax.set_title(f'{label} Transit Upper Limits', fontsize=18, pad=15)

    handles = [plt.Line2D([0], [0], marker='o', color='w', label=grb,
                          markerfacecolor=col, markersize=8)
               for grb, col in zip(names[valid_mask], color_hex[:n_valid_points])]

    if handles:
        ax.legend(handles, names[valid_mask], loc='upper center', bbox_to_anchor=(0.5, -0.25),
                  fontsize=9, ncol=6, frameon=False, title=f'{label} Transit')

    fig.tight_layout()
    if save_path_png:
        plt.savefig(save_path_png, dpi=600, bbox_inches='tight')
    if save_path_pdf:
        plt.savefig(save_path_pdf, dpi=600, bbox_inches='tight')
    plt.close(fig)

# ===========================
# 4. GENERADOR DE GRÁFICAS
# ===========================
def generate_upper_limits_plots(config):
    spectral_index = config['spectral_index']
    plots_dir = config['plots_dir']
    os.makedirs(plots_dir, exist_ok=True)

    df_1st = DATAFRAME_generator(config['Franceschini_1st'], config['Gilmore_1st'], config['GRBsINFO'])
    df_2nd = DATAFRAME_generator(config['Franceschini_2nd'], config['Gilmore_2nd'], config['GRBsINFO'])

    def integrate(df, bound_fra, bound_gil):
        E1 = np.full(len(df), 100)
        E2 = np.full(len(df), 1e5)
        ul_fra = non_to_int2(E1, E2, df[bound_fra], spectral_index, 1)
        ul_gil = non_to_int2(E1, E2, df[bound_gil], spectral_index, 1)
        return ul_fra, ul_gil

    ul_fra_1, ul_gil_1 = integrate(df_1st, 'upperBound_fra', 'upperBound_gil')
    ul_fra_2, ul_gil_2 = integrate(df_2nd, 'upperBound_fra', 'upperBound_gil')

    cmaps = [plt.cm.tab20, plt.cm.Set1, plt.cm.Set2, plt.cm.Set3, plt.cm.Paired, plt.cm.Accent, plt.cm.Dark2]
    colors = np.vstack([cmap(np.linspace(0, 1, cmap.N)) for cmap in cmaps])
    color_hex = [matplotlib.colors.rgb2hex(c) for c in colors][:max(len(df_1st), len(df_2nd))]

    plot_ul(df_1st['Name'], df_1st['Dec'], ul_fra_1, ul_gil_1, color_hex,
            label='First',
            save_path_png=os.path.join(plots_dir, 'upper_limits_first.png'),
            save_path_pdf=os.path.join(plots_dir, 'upper_limits_first.pdf'))

    plot_ul(df_2nd['Name'], df_2nd['Dec'], ul_fra_2, ul_gil_2, color_hex,
            label='Second',
            save_path_png=os.path.join(plots_dir, 'upper_limits_second.png'),
            save_path_pdf=os.path.join(plots_dir, 'upper_limits_second.pdf'))

    print("\n✅ Upper limits plots saved in:", plots_dir)

generate_upper_limits_plots(config)



GRB150101641 3.796478080424418e-09 3.840918459292262e-09
GRB150110923 1.6633398947678893e-10 9.142020795670841e-11
GRB150120123 9.71339709540027e-08 1.2633764849572897e-07
GRB151229285 1.3459086171404294e-09 4.786863666622093e-10
GRB160612842 3.187010027379696e-07 4.1900928646824685e-07
GRB160624477 1.0348259650655189e-08 1.3776517449031753e-08
GRB160821937 4.183744239129919e-08 4.34245987794365e-08
GRB170318644 8.062754451737477e-11 4.2535791202079606e-11
GRB170403583 3.8028267059769685e-11 2.2855051989177103e-11
GRB170708046 1.199890229431798e-10 6.920001852278623e-11
GRB170803729 3.7012486971361807e-10 2.964808133040474e-10
GRB170816599 1.4855783792965117e-10 8.253213218313955e-11
GRB170817529 9.967342117502237e-10 4.717028785544051e-10
GRB170826369 7.808809429635511e-11 5.1360380720122985e-11
GRB180204109 1.4284407493235692e-10 8.189726962788463e-11
GRB180418281 1.1617984761165025e-10 6.793029341227637e-11
GRB180715755 2.0379088023682914e-10 9.967342117502236e-11
GRB180718082 1.377

In [10]:
import pandas as pd

# 1) Define the column names
cols = [
    'GRB',        # e.g. GRB170816599
    'Model',      # e.g. PowerLaw
    'Prefactor',  # e.g. 1.0e-10
    'Index',      # e.g. 2.07
    'Param1',     # e.g. 2.78
    'Param2',     # e.g. 1.67
    'Value',      # e.g. 3.07e-12
    'Lower',      # e.g. -3.07e-12
    'Upper',      # e.g. +3.18e-12
    'Zero',       # e.g. 0.00e+00
    'High'        # e.g. 6.25e-12
]
FILE='/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/files/PSF_0.3/alfa=2.07/UpperLimit_1_Franceschini08.csv'
# FILE='/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/files/PSF_0.3/alfa=2.07/UpperLimit_1_Gilmore12Fiducial.csv'
# 2) Read with a regex separator: split on commas *or* any whitespace
df_2 = pd.read_csv(
    FILE,
    sep=r'\s*,\s*|\s+',
    engine='python',
    header=None,
    names=cols
)
# 2) Read with a regex separator: split on commas *or* any whitespace
df = pd.read_csv(
    FILE,
    sep=r'\s*,\s*|\s+',
    engine='python',
    header=None,
    names=cols)

# 3) Quick sanity check
# print(df.head())
# print(df.dtypes)
df.drop_duplicates(inplace=True)
df.sort_values(by='GRB',ignore_index=True)
# 1) Lista de GRBs de interés
target_grbs = [
    "GRB141205337", "GRB150101270", "GRB150101641", "GRB150110923", "GRB150819440",
    "GRB150922234", "GRB151229285", "GRB160612842", "GRB160624477", "GRB160714097",
    "GRB160726065", "GRB170206453", "GRB170222209", "GRB170318644", "GRB170325331",
    "GRB170403583", "GRB170708046", "GRB170803729", "GRB170816599", "GRB170817529",
    "GRB170826369", "GRB171007498", "GRB180204109", "GRB180402406", "GRB180418281",
    "GRB180715755", "GRB180718082", "GRB180805543", "GRB181125371", "GRB190427190",
    "GRB190515190", "GRB191031891", "GRB200605762", "GRB200623138", "GRB201008443",
    "GRB201214672", "GRB201221963", "GRB210323918", "GRB210618072", "GRB210827416",
    "GRB211024065", "GRB220412713", "GRB220418720", "GRB220511571", "GRB220617772",
    "GRB221120895", "GRB230228244", "GRB230512269", "GRB230812790"
]
GRBs_2nt=['GRB150819440',
'GRB160714097',
'GRB170206453',
'GRB171007498',
'GRB180402406',
'GRB191031891',
'GRB200605762',
'GRB200623138',
'GRB210618072']
target_grbs=[x for x in target_grbs if x not in GRBs_2nt]
# 2) Creamos una columna booleana indicando si el GRB está en la lista
df['in_list'] = df['GRB'].isin(target_grbs)

# 3) Filtramos sólo los que sí están
df_in_list = df[df['in_list']].copy()

# 4) (Opcional) Para ver cuáles GRBs de tu lista NO aparecieron en el DataFrame:
present = set(df['GRB'].unique()) & set(target_grbs)
missing = set(target_grbs) - present
print("GRBs presentes en el DataFrame:", sorted(present))
print('Amount of GRBs:',len(present))
print("GRBs faltantes:", sorted(missing))

# Ahora df_in_list contiene únicamente las filas cuyos GRB están en tu lista
print(df_in_list.head())
df

GRBs presentes en el DataFrame: ['GRB141205337', 'GRB150101270', 'GRB150101641', 'GRB150110923', 'GRB150922234', 'GRB151229285', 'GRB160612842', 'GRB160624477', 'GRB160726065', 'GRB170222209', 'GRB170318644', 'GRB170325331', 'GRB170403583', 'GRB170708046', 'GRB170803729', 'GRB170816599', 'GRB170817529', 'GRB170826369', 'GRB180204109', 'GRB180418281', 'GRB180715755', 'GRB180718082', 'GRB180805543', 'GRB181125371', 'GRB190427190', 'GRB190515190', 'GRB201008443', 'GRB201214672', 'GRB201221963', 'GRB210323918', 'GRB210827416', 'GRB211024065', 'GRB220412713', 'GRB220418720', 'GRB220511571', 'GRB220617772', 'GRB221120895', 'GRB230228244', 'GRB230512269', 'GRB230812790']
Amount of GRBs: 40
GRBs faltantes: []
            GRB     Model     Prefactor  Index  Param1  Param2         Value  \
0  GRB141205337  PowerLaw  1.000000e-10   2.07    1.65    1.28  8.010000e-12   
1  GRB150101270  PowerLaw  1.000000e-10   2.07    1.68    1.30  6.940000e-12   
2  GRB150101641  PowerLaw  1.000000e-10   2.07   

,GRB,Model,Prefactor,Index,Param1,Param2,Value,Lower,Upper,Zero,High,in_list
0,GRB141205337,PowerLaw,1.000000e-10,2.07,1.65,1.28,8.010000e-12,-8.010000e-12,8.450000e-12,0.000000e+00,1.650000e-11,True
1,GRB150101270,PowerLaw,1.000000e-10,2.07,1.68,1.30,6.940000e-12,-6.940000e-12,7.700000e-12,0.000000e+00,1.460000e-11,True
2,GRB150101641,PowerLaw,1.000000e-10,2.07,1.96,1.40,2.220000e-10,-1.810000e-10,1.810000e-10,4.090000e-11,4.030000e-10,True
3,GRB150110923,PowerLaw,1.000000e-10,2.07,3.34,1.83,7.290000e-12,-6.970000e-12,6.970000e-12,3.210000e-13,1.430000e-11,True
4,GRB150120123,PowerLaw,1.000000e-10,2.07,3.70,1.92,6.310000e-09,-3.310000e-09,3.310000e-09,3.000000e-09,9.620000e-09,False
5,GRB150922234,PowerLaw,1.000000e-10,2.07,4.60,2.14,7.550000e-12,-7.550000e-12,7.550000e-12,0.000000e+00,1.510000e-11,True
6,GRB151228129,PowerLaw,1.000000e-10,2.07,0.15,0.39,1.010000e-11,-1.010000e-11,2.830000e-11,0.000000e+00,3.840000e-11,False
7,GRB151229285,PowerLaw,1.000000e-10,2.07,1.82,1.35,6.390000e-11,-6.250000e-11,6.250000e-11,1.430000e-12,1.260000e-10,True
8,GRB160612842,PowerLaw,1.000000e-10,2.07,0.52,0.72,1.050000e-10,-1.050000e-10,1.680000e-10,0.000000e+00,2.730000e-10,True
9,GRB160624477,PowerLaw,1.000000e-10,2.07,0.82,0.91,4.540000e-10,-4.540000e-10,5.020000e-10,0.000000e+00,9.570000e-10,True


40

In [12]:

# Función general para leer archivos de upper limits y graficar por separado cada tránsito

import matplotlib.pyplot as plt
import numpy as np
import matplotlib

def read_upper_limit_csv(filepath):
    """Lee un archivo de upper limits sin encabezado y separados por espacio"""
    column_names = ['Name', 'Model', 'Norm', 'Index', 'TS', 'Dec', 'RA', 'UL', 'UL_minus']
    df = pd.read_csv(filepath, delim_whitespace=True, header=None, names=column_names)
    df['Name'] = df['Name'].str.strip().str.replace(',', '')
    return df

def plot_upper_limits_per_transit(df_fra, df_gil, label='First', save_path=None):
    """Grafica los upper limits para un solo tránsito (Franceschini vs Gilmore)"""

    names = df_fra['Name']
    dec = df_fra['Dec']
    ul_fra = df_fra['UL']
    ul_gil = df_gil['UL']

    # Estética
    cmaps = [plt.cm.tab20, plt.cm.Set1, plt.cm.Set2, plt.cm.Set3, plt.cm.Paired, plt.cm.Accent, plt.cm.Dark2]
    colors = np.vstack([cmap(np.linspace(0, 1, cmap.N)) for cmap in cmaps])
    color_hex = [matplotlib.colors.rgb2hex(c) for c in colors][:len(names)]

    fig, ax = plt.subplots(figsize=(12, 7))
    plt.rcParams.update({
        "text.usetex": False,
        "font.family": "serif",
        "font.size": 14
    })

    legend_labels = []

    for grb, ul_f, ul_g, d, col in zip(names, ul_fra, ul_gil, dec, color_hex):
        if np.isnan(ul_f) or np.isnan(ul_g) or np.isnan(d):
            continue

        ax.errorbar(x=d, y=ul_g, yerr=ul_g / 10, uplims=True,
                    color=col, linewidth=1.6, capsize=4, linestyle='solid')
        ax.fill_between([d - 0.4, d + 0.4], [ul_f, ul_f], [ul_g, ul_g],
                        color=col, alpha=0.5)
        legend_labels.append(grb)

    ax.set_yscale('log')
    ax.set_xlabel('Declination [deg]', fontsize=16, labelpad=10)
    ax.set_ylabel('Integrated Upper Limit (Φ) [erg cm$^{-2}$ s$^{-1}$]', fontsize=14, labelpad=10)
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.grid(True, which='both', linestyle=':', linewidth=0.7, alpha=0.7)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    if legend_labels:
        ax.legend(legend_labels, loc='upper center', bbox_to_anchor=(0.5, -0.25),
                  fontsize=9, ncol=6, frameon=False, title=f'{label} Transit')

    fig.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
    plt.close(fig)

def run_upper_limit_plot_pipeline(
    file_fra_1, file_gil_1, file_fra_2, file_gil_2,
    output_png_1='upper_limits_first.png',
    output_png_2='upper_limits_second.png'
):
    """Lee los archivos y genera los plots separados para cada tránsito"""
    df1_fra = read_upper_limit_csv(file_fra_1)
    df1_gil = read_upper_limit_csv(file_gil_1)
    df2_fra = read_upper_limit_csv(file_fra_2)
    df2_gil = read_upper_limit_csv(file_gil_2)

    plot_upper_limits_per_transit(df1_fra, df1_gil, label='First', save_path=output_png_1)
    plot_upper_limits_per_transit(df2_fra, df2_gil, label='Second', save_path=output_png_2)

    print("✅ Upper limit plots saved:")
    print(" - First Transit :", output_png_1)
    print(" - Second Transit:", output_png_2)

